<a href="https://colab.research.google.com/github/jhauser6129/PEASS-ng/blob/master/Josh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

def find_pid_by_port(port):
    """Find the PID of the process using the specified port."""
    try:
        # Run the netstat command to find the PID
        netstat_output = subprocess.check_output(
            ['netstat', '-ano'], stderr=subprocess.STDOUT, text=True
        )

        # Parse the output
        for line in netstat_output.splitlines():
            if f":{port} " in line:
                parts = line.split()
                pid = parts[-1]
                return int(pid)
        return None
    except subprocess.CalledProcessError as e:
        print(f"Error executing netstat: {e}")
        return None

def kill_process_by_pid(pid):
    """Terminate the process with the specified PID."""



In [1]:
import socket
from concurrent.futures import ThreadPoolExecutor

def scan_port(ip, port):
    """Attempt to connect to a specific port on a given IP address."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.settimeout(1)  # Set timeout to 1 second
        try:
            sock.connect((ip, port))
            return port, True
        except (socket.timeout, ConnectionRefusedError):
            return port, False

def scan_ports(ip, start_port, end_port, max_workers=100):
    """Scan a range of ports on the given IP address."""
    open_ports = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_port = {executor.submit(scan_port, ip, port): port for port in range(start_port, end_port + 1)}
        for future in future_to_port:
            port, is_open = future.result()
            if is_open:
                open_ports.append(port)
    return open_ports

if __name__ == "__main__":
    target_ip = input("Enter the IP address to scan: ")
    start_port = int(input("Enter the start port: "))
    end_port = int(input("Enter the end port: "))

    print(f"Scanning {target_ip} for open ports from {start_port} to {end_port}...")
    open_ports = scan_ports(target_ip, start_port, end_port)

    if open_ports:
        print(f"Open ports on {target_ip}: {', '.join(map(str, open_ports))}")
    else:
        print(f"No open ports found on {target_ip} in the range {start_port}-{end_port}.")


Enter the IP address to scan: 192.168.1.1
Enter the start port: 1
Enter the end port: 64000
Scanning 192.168.1.1 for open ports from 1 to 64000...
No open ports found on 192.168.1.1 in the range 1-64000.
